# Segmenting and Clustering Neighbourhoods in Toronto

### Section: #1 - Explore and cluster neighbourhoods in Toronto

Capstone project - Battle of the Neighbourhoods. This project will compare suburbs and will determine similarities based on clustering techniques using location data services.

This project uses web scraping techniques to retrieve data from the Canadian Postal Code's Wikipedia page.

The data is then acquired and cleansed in preparation for clustering.

We import a file that contains the geospatial loctions which we then merge with the post code data which enables us to visualise the data over a map of the area.

We then cluster and plot the data over the map.

The clustering is carried out by K Means and the clusters are plotted using the Folium Library.

We map the data across Toronto and then focus/cluster the data in on boroughs containing the name 'Toronto'.

Install and import required libraries..

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Setup the reference to the Postal Codes of Canada wiki page..

Read in the web page and define a Beautiful Soup object for manipulation

In [ ]:
canadian_postcodes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
canadian_postcodes = requests.get(canadian_postcodes_url).text
soup = BeautifulSoup(canadian_postcodes, "html5lib")


lets find the tables within the web page

In [ ]:
table_contents = []
canadian_postcodes_table = soup.find("table")

Lets take a look at the raw html table..

In [ ]:
canadian_postcodes_table

we're going to wrangle the data into a dataframe consisting of 3 columns..

PostalCode, Borough, Neighborhood

And we're going to remove those cells that contain Not Assigned, merge neighborhoods into a single postal code area and remove the duplicate postal codes

In [ ]:
for row in canadian_postcodes_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass #ignore these ones
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

#merge neighborhoods
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#remove duplicate values - some postal code area have multiple neighbourhoods i.e.M5A
postcode_data = df.drop_duplicates()

#check de-duped
#postcode_data.loc[df['PostalCode'] == 'M5A']



view the data

In [ ]:
postcode_data

Answer for Section 1 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

print the number of rows in the data

In [ ]:
postcode_data.shape

###Section: #2 - combining the Postal Code data with Geospatial data


Reading the Geospatial_Coordinates csv file from notebook project assets.... tried using the Geocoder package, but received errors, so CSV it is...

this cell will be hidden, however the data is read into a dataframe named geospatial_data and it will be referenced later in the notebook

In [ ]:
# The code was removed by Watson Studio for sharing.

Lets review the Geospatial data

In [ ]:
geospatial_data

print the number of rows in the data

In [ ]:
geospatial_data.shape

We're going to merge (or join) the post code data and the geospatial data so that we can link up the latitudes and longitudes for the neighbourhoods in Canada.

In [ ]:
geospatial_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
merged_geospatial_data = pd.merge(postcode_data,geospatial_data,on='PostalCode')


Answer for Section 2 - output the dataframe data as per the example

In [ ]:
merged_geospatial_data.head(12)

###Section: #3 - Explore and cluster the neighborhoods in Toronto


lets do some quick analysis around the data, how many boroughs and neighborhoods are there in the data..

In [ ]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(merged_geospatial_data['Borough'].unique()),
        merged_geospatial_data.shape[0]
    )
)


Use geopy library to get the lattitude and longitude of Toronto

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))



lets visualise the data over a map of Toronto, Ontario

In [ ]:
toronto_data = merged_geospatial_data

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Lets segment and cluster only the neighborhoods that contain "toronto" in their name

In [ ]:
toronto_boroughs_data = merged_geospatial_data[merged_geospatial_data['Borough'].str.contains('Toronto',regex=False)]
toronto_boroughs_data


In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto_boroughs = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_boroughs_data['Latitude'], toronto_boroughs_data['Longitude'], toronto_boroughs_data['Borough'], toronto_boroughs_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_boroughs)  
    
map_toronto_boroughs

define K Means cluster on the boroughs of Toronto

In [ ]:
k=5
toronto_clustering = toronto_boroughs_data.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
toronto_boroughs_data.insert(0, 'Cluster Labels', kmeans.labels_)

view the clustered table

In [ ]:
toronto_boroughs_data

visualise the clusters over the map

In [ ]:
# create map
map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_boroughs_data['Latitude'], toronto_boroughs_data['Longitude'], toronto_boroughs_data['Neighborhood'], toronto_boroughs_data['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto_clusters)
       
map_toronto_clusters